# Predicting the Outcome of NBA Games using Recent Team Performance and Elo

In this notebook, we will use the dataset we processed from the feature engineering notebook to train various models on predicting the outcome of games.

The data includes a team's and opponent's elo rating and performance from the last 10 games.

## Read the data